In [1]:
# ------------------------------------------------------------------------------
## [Description]
### This function calculates the fold-change and p-value of two groups.
## [Input]
### dataframe -> the grouped data; groupVector_a/b --> a vector of groupA that represent the column index;
## [Output]
### returns the input dataframe with two additional(new) columns: 'fold_change', 'p_value'
## [Details]
### When calculating the p-value, this function use apply() to get the results
## [Notes]
### This function is faster than get_Ttest_Result_2()
# ------------------------------------------------------------------------------
get_Ttest_Result_2 <- function(dfrm, groupVector_a, groupVector_b){
    # Get the average value of sample group A (non-smokers) & group B (smokers)
    non_smoker_mean <- apply(dfrm[, groupVector_a], 1, mean)
    smoker_mean <- apply(dfrm[, groupVector_b], 1, mean)
    # Calculate the fold-change
    fold_change <- smoker_mean / non_smoker_mean
    fold_change_dfrm <- as.data.frame(fold_change)

    # Use apply() to apply t.test and get results of p-value
    ## Use tempFunction to get p-value
    p_value <- apply(dfrm[1:(dim(dfrm)[1]-1),], 1, function(x) t.test(as.numeric(x[groupVector_a]),as.numeric(x[groupVector_b]))$p.value)
    p_value <- append(p_value, NA) # Handle the NA value
    p_value_dfrm <- as.data.frame(p_value)

    # Add two new columns to the input dfrm
    dfrm <- cbind(dfrm, fold_change_dfrm, p_value_dfrm)
    # Return Value
    return(dfrm)
}
                     

dfrm <- read.csv('C:\\Users\\Nature\\Desktop\\M_RLanguage\\Lab_6\\GSE5056_series_matrix.txt',
                header = TRUE, sep='\t', row.names=1)
dfrm_new = na.omit(get_Ttest_Result_2(dfrm, 1:18, 19:44))

In [2]:
co_p <- 2 # cutoff of p_value: 0.01 (-log10(0.01) = 2)
co_f <- 0.35 # cutoff of fold_change: +-1.274561(log2(cutoff) = +-0.35)
dfrm_new$Expression = "Normal"
dfrm_new[(dfrm_new$fold_change > 2^co_f) & (dfrm_new$p_value < 10^-co_p),]['Expression']='Up'
dfrm_new[(dfrm_new$fold_change < 2^-co_f) & (dfrm_new$p_value < 10^-co_p),]['Expression']='Down'

In [36]:
pdf("Boxplot_Plot_of_GSE5056_zzf_0605.pdf")
# 底部 左部 顶部 右边
par(mfrow=c(1,2),mar=c(8,9,6,3),bg='#FFFFF0',tcl=0,cex.axis=0.5,mex=0.24)
boxplot(log2(dfrm_new[dfrm_new$Expression == 'Up',][,1:44])+1,
        horizontal=TRUE, cex=0.5,las=2,col=topo.colors(44),border="#708090", pch=16,
       xlab="log2(Up Expression)+1",yaxt = "n",ylab="Example")
boxplot(log2(dfrm_new[dfrm_new$Expression == 'Down',][,1:44])+1,
        horizontal=TRUE, cex=0.5,las=2,col=topo.colors(44),border="#708090",pch=16,
       xlab="log2(Down Expression)+1")
mtext("BoxPlot of Expression", side = 3, line = -4, outer = TRUE)
# mtext("Example", side = 3, line = -55, outer = TRUE,)
dev.off()

png 
  2